In [1]:
import numpy as np
import numpy.linalg as LA

In [2]:
single = np.float32
double = np.float64
np.set_printoptions(linewidth=120)

# Question 1

In [3]:
def qr(A, b):
    q, r = LA.qr(A)
    p = q.T.dot(b)
    cond = LA.cond(A)
    x = LA.inv(r).dot(p)
    err = LA.norm(A.dot(x) - b, ord=2)
    print('k2(A) : %.2f' % cond)
    print('Error : %.2e' % err)
    return x

def ne(A, b):
    AT = A.T
    ATA = AT.dot(A)
    cond = LA.cond(ATA)
    p = AT.dot(b)
    x = LA.solve(ATA, p)
    err = LA.norm(A.dot(x) - b, ord=2)
    print("k2(A'A) : %.2f" % cond)
    print('Error : %.2e' % err)
    return x

def ls(A, b):
    Ad = A.astype(double)
    bd = b.astype(double)
    x = LA.lstsq(Ad, bd)[0]
    return x

def compare(xqr, xne, xls):
    labels = ['QR', 'NE']
    errs = [LA.norm(xqr - xls, 2), LA.norm(xne - xls, 2)]
    for label, err in zip(labels, errs):
        print('%s error : %.2e' % (label, err))
    print('Ratio : %.0f' % (max(errs)/min(errs)))
    
def q1(A, b):
    print('i.')
    xqr = qr(A, b)
    print('ii.')
    xne = ne(A, b)
    print('iii.')
    xls = ls(A, b)
    compare(xqr, xne, xls)

In [4]:
t = np.arange(1.11, 1.118, 0.001)
b = np.array([2.1101, 2.1111, 2.1121, 2.1130, 2.1140, 2.1150, 2.1160, 2.1170, 2.1180]).astype(single)
m = len(b)
A = np.vstack([np.ones(m), t]).T.astype(single)

In [5]:
q1(A, b)

i.
k2(A) : 867.93
Error : 8.06e-05
ii.
k2(A'A) : 760117.50
Error : 1.75e-04
iii.
QR error : 2.08e-05
NE error : 3.01e-02
Ratio : 1449


## b)

In [6]:
A[:, 1] = 100.0*(t - 1.11)

In [7]:
q1(A, b)

i.
k2(A) : 4.53
Error : 8.06e-05
ii.
k2(A'A) : 20.52
Error : 8.07e-05
iii.
QR error : 3.37e-07
NE error : 1.90e-06
Ratio : 6


---

# Question 3

## a)

In [8]:
def iszero(x):
    return np.isclose(x, 0, rtol=1e-10)

def mkmat(c, s):
    return np.array([[c, -s], [s, c]])

def symmetric_transform(A):
    (w, x), (y, z) = A
    if iszero(w + z):
        c = 0
        s = 1
    else:
        t = (x - y)/(w + z)
        c = 1/np.sqrt(1 + t**2)
        s = t*c
    return mkmat(c, s)

def diagonalize(A):
    (w, x), (y, z) = A
    if iszero(y):
        c = 1
        s = 0
    else:
        rho = (z - w)/(2*y)
        t = np.sign(rho)/(np.abs(rho) + np.sqrt(rho**2 + 1))
        c = 1/np.sqrt(1 + t**2)
        s = t*c
    return mkmat(c, s)

def jacobi(A):
    """
    Compute Jacobi rotation on A.
    
    Args:
        A (2x2 ndarray)
    
    Returns:
        Jl, Jr (2x2 ndarray) : Left and right Jacobi rotators
    """
    # Step 1
    cs = symmetric_transform(A)
    Asym = cs.dot(A)
    # Step 2
    cs2 = diagonalize(Asym)
    Jl = cs2.dot(cs)
    Jr = cs2.T
    return Jl, Jr

In [9]:
# Test
for i in range(5):
    A = np.random.rand(2, 2)
    Jl, Jr = jacobi(A)
    Ap = Jl.dot(A).dot(Jr)
    assert iszero(Ap[0, 1])
    assert iszero(Ap[1, 0])
    assert np.isclose(LA.norm(A), LA.norm(Ap), rtol=1e-10)

No Assertion Errors. Very good.

## b, c)

In [10]:
def givens(y):
    """
    Compute Givens rotation
    
    Args:
        y (2x1 ndarray)
        
    Returns:
        G (2x2 ndarray)
    """
    a, b = y
    if iszero(b):
        c = 1
        s = 0
    else:
        r = LA.norm(y)
        s = b/r
        c = a/r
    return mkmat(c, s).T

In [11]:
def mk_off(m, n):
    """
    Make off-diagonal norm function based on shape.
    
    It is assumed that m >= n.
    """
    diag_indices = (range(n), range(n))
    mask = np.ones((m, n), dtype=bool)
    mask[diag_indices] = False
    def off(A):
        return LA.norm(A[mask])
    return off

def svd(A, tol):
    """Kogbetliantz algorithm. Overwrites A."""
    m, n = A.shape
    U = np.eye(m, dtype=float)
    V = np.eye(n, dtype=float)
    eps = tol*LA.norm(A, ord='fro')
    off = mk_off(m, n)
    max_iter = 100000
    for it in range(max_iter):
        for i in range(n):
            for j in range(i + 1, n):
                selector = [((i, i), (j, j)), ((i, j), (i, j))]
                Jl, Jr = jacobi(A[selector])
                Ql, Qr = np.eye(m, dtype=float), np.eye(n, dtype=float)
                Ql[selector], Qr[selector] = Jl, Jr
                A = Ql.dot(A).dot(Qr)
                U = U.dot(Ql)
                V = V.dot(Qr)
            for j in range(n, m):
                selector = [((i, i), (j, j)), ((i, j), (i, j))]
                Ql = np.eye(m, dtype=float)
                G = givens(A[(i, j), (i, i)])
                Ql[selector] = G
                A = Ql.dot(A)
                U = U.dot(Ql)
            
        if off(A) < eps:
            break
    else:
        print("Max number of iterations")
    S = A
    return U, V, S

In [12]:
B = np.array([
    [-1.1, -6, -11],
    [2, 7, 12],
    [-3, -8, -13],
    [4, 9, 14],
    [-5, -10, -15]
], dtype=double)

In [13]:
U, V, S = svd(B, 1e-10)
_, n = S.shape
singvals = sorted(abs(S[range(n), range(n)]), reverse=True)
singvals

[35.134394975254473, 2.4049094338397534, 0.026463896961604845]

In [14]:
_, exact_singvals, _ = LA.svd(B)

In [15]:
exact_singvals

array([  3.51343950e+01,   2.40490943e+00,   2.64638970e-02])

In [16]:
err = abs(singvals - exact_singvals)
err/LA.norm(A)

array([  7.73861369e-15,   2.41831678e-15,   4.68548876e-15])

If we suppose that the values given by SciPy are the exact ones, then we can say that our algorithm might be stable, due to the singular values obtained being of order $O(u)\cdot||A||$. General stability can only be proven mathematically.

## e)

In [17]:
A = np.array([
    [-1, -6, -11],
    [2, 7, 12],
    [-3, -8, -13],
    [4, 9, 14],
    [-5, -10, -15]
], dtype=double)

U, s, V = LA.svd(A)

The third column is clearly a multiple of the other two

In [18]:
col3 = 2*A[:, 1] - A[:, 0]
col3

array([-11.,  12., -13.,  14., -15.])

In [19]:
col3 == A[:, 2]

array([ True,  True,  True,  True,  True], dtype=bool)

In [20]:
U

array([[-0.35455706, -0.68868664,  0.4974389 ,  0.15945456, -0.35655124],
       [ 0.39869637,  0.37555453,  0.15918469,  0.33532577, -0.74981122],
       [-0.44283568, -0.06242242, -0.69972506,  0.52744566, -0.17940435],
       [ 0.486975  , -0.2507097 ,  0.1127514 ,  0.68673226,  0.46441999],
       [-0.53111431,  0.56384181,  0.47422225,  0.33515781,  0.25056436]])

In [21]:
s

array([  3.51272233e+01,   2.46539670e+00,   8.10792259e-16])

In [22]:
V

array([[ 0.20166491,  0.5168305 ,  0.83199609],
       [-0.89031713, -0.25733163,  0.37565388],
       [ 0.40824829, -0.81649658,  0.40824829]])

We're probably going to run into trouble with that third singular value: it's close to zero, but not quite. 

## f)

In [23]:
sinv = 1/s
Sinv = np.zeros_like(A).T
Sinv[range(3), range(3)] = sinv
Sinv

array([[  2.84679489e-02,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.05614237e-01,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.23336155e+15,   0.00000000e+00,   0.00000000e+00]])

In [24]:
G = V.T.dot(Sinv).dot(U.T)
G

array([[  2.50469312e+14,   8.01523159e+13,  -3.52323983e+14,   5.67723319e+13,   2.38779320e+14],
       [ -5.00938623e+14,  -1.60304632e+14,   7.04647967e+14,  -1.13544664e+14,  -4.77558639e+14],
       [  2.50469312e+14,   8.01523159e+13,  -3.52323983e+14,   5.67723319e+13,   2.38779320e+14]])

Woah. 

Test $AGA = A$

In [25]:
normA = LA.norm(A)
LA.norm((A.dot(G).dot(A) - A))/normA

1.117526700987209

Test $GAG = G$

In [26]:
normG = LA.norm(G)
LA.norm((G.dot(A).dot(G) - G))/normG

0.30618621784789773

Test $(AG)^T = AG$

In [27]:
AG = A.dot(G)
normAG = LA.norm(AG)
LA.norm(AG.T - AG)/normAG

1.4196178374164063

Test $(GA)^T = GA$

In [28]:
GA = G.dot(A)
normGA = LA.norm(GA)
LA.norm(GA.T - GA)/normGA

1.3562969981821595

Overall, G is a terrible approximation.

## g)

In [29]:
b = np.array([5, 4, 3, 2, 1])
b.resize((len(b), 1))

In [30]:
x = G.dot(b)

In [31]:
LA.norm(x)

2126911183522930.8

$||x||$ is huge!

In [32]:
LA.norm(b - A.dot(x))

8.8881944173155887

In [33]:
x_exact = LA.lstsq(A, b)[0]
LA.norm(x_exact)

0.83857021172946622

In [34]:
LA.norm(b - A.dot(x_exact))

7.0427267446636028

Not only did we fail to find the solution to $||Ax - b||$, but we clearly did not minimize $||x||$. 

## h)

Clearly, our Moore-Penrose inverse was not good. However, let's see what happens if we realize that the third column is redundant. 

In [35]:
A = A[:, :2]
A

array([[ -1.,  -6.],
       [  2.,   7.],
       [ -3.,  -8.],
       [  4.,   9.],
       [ -5., -10.]])

In [36]:
U, s, V = LA.svd(A)
sinv = 1/s
Sinv = np.zeros_like(A).T
Sinv[range(2), range(2)] = sinv
G = V.T.dot(Sinv).dot(U.T)
G

array([[ 0.36, -0.2 ,  0.04,  0.12, -0.28],
       [-0.16,  0.1 , -0.04, -0.02,  0.08]])

In [37]:
normA = LA.norm(A)
LA.norm((A.dot(G).dot(A) - A))/normA

1.8247212710005693e-16

In [38]:
x = G.dot(b)

In [39]:
LA.norm(x)

1.1818629362155337

In [40]:
LA.norm(b - A.dot(x))

7.0427267446636028

Much better! So, not checking for very small singular values leads to problems. However, keeping that in mind and flushing out the computational artifacts, the right answer can be obtained.